In [1]:
#pip install textblob
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  

In [2]:
api_key="9ec526b31a214d80bfeb5aa83451239c"
type_of_news="everything" ## everything || top-headlines
date_from="2019-07-26"
querying_keyword="Iron ore"
pageSize="100"
source="Reuters"
news_feed_url = ('https://newsapi.org/v2/'+type_of_news+'?q='+querying_keyword+'&from='+date_from+'&sortBy=popularity&sources='+source+'&pageSize='+pageSize+'&apiKey='+api_key)

In [3]:
def get_data(news_feed_url):
    response = requests.get(news_feed_url)
    return response.json()

a=get_data(news_feed_url)

In [4]:
#print(a)
news_data=json_normalize(a['articles'])
type(news_data)

pandas.core.frame.DataFrame

In [5]:
date_stamp=news_data["publishedAt"].str.split("T",n=1,expand=True)
news_data["published_date"]=date_stamp[0]
news_data["published_time"]=date_stamp[1].str.split("Z",n=1,expand=True)[0]


In [6]:
news_data.head(1)

,author,content,description,publishedAt,source.id,source.name,title,url,urlToImage,published_date,published_time
0,Reuters Editorial,"BEIJING (Reuters) - China’s monthly crude steel output fell for a second straight month in July, official data showed on Wednesday, as steel mills trimmed output amid heightened environmental measures and record raw material prices. \r\nThe world’s top steelmak… [+1672 chars]","China's monthly crude steel output fell for a second straight month in July, official data showed on Wednesday, as steel mills trimmed output amid heightened environmental measures and record raw material prices.",2019-08-14T02:38:59Z,reuters,Reuters,"China's July steel output eases on environmental curbs, shrinking margins",https://www.reuters.com/article/us-china-economy-output-steel-idUSKCN1V4062,https://s3.reutersmedia.net/resources/r/?m=02&d=20190814&t=2&i=1418702817&w=1200&r=LYNXNPEF7D05J,2019-08-14,02:38:59


In [7]:
news_data.to_csv('data_reuters_iron.csv')

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [19]:
from IPython.display import FileLink
FileLink(r'data_reuters_iron.csv')

/kaggle/working/data_reuters_iron.csv

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
!pip install rouge
from rouge import Rouge
rouge = Rouge()
import string
import networkx as nx

from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# Function to tokenize sentences properly
def clean_tokenize(full_text):
    full_text = re.sub(r"\\'", "'", full_text)        # few modifications in text
    full_text = re.sub(r"U.S.", "US", full_text)
    #full_text = re.sub(r"[^a-zA-Z0-9]", " ", full_text)
    full_text = full_text.replace('\n','')
  
  
    tokenized = nltk.sent_tokenize(full_text)         # nltk tokenizer
  
    for sentence in tokenized:                        # identifying correct positions for tokenization
        x=re.findall(r'\w+[.?!][A-Z]+', sentence)
        all_delm = []
        punctuation = [".","!","?"]
        for punct in punctuation:
            for occurrence in x:
                try:
                    idx1 = occurrence.index(punct)
                    idx2 = sentence.index(occurrence)
                    punct_idx = idx1+idx2
                    all_delm .append(punct_idx)
                except:
                    continue
          
        all_delm.sort()
        good_tok = []
        lower_idx = 0
        higher_idx = 0
    
        for i in range(len(all_delm)):                  # creating list of properly tokenized text
            if i!=0:
                lower_idx = all_delm[i-1]+1
            higher_idx = all_delm[i]+1
            good_tok.append(sentence[lower_idx:higher_idx])
        good_tok.append(sentence[higher_idx:])
    
        sent_idx = tokenized.index(sentence)            
        for i in range(len(good_tok)):
            tokenized.insert(sent_idx+i+1, good_tok[i])
        tokenized.pop(sent_idx)
  
    #print (tokenized)
    return tokenized

In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
embeddings_index = {}
f = open('../input/glove840b300dtxt/glove.840B.300d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    embeddings_index[word] = coefs
f.close()

print('GloVe data loaded')

In [ ]:
# convert sentence to vector
def convert_to_vectors(clean_sentences):
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([embeddings_index.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((300,))
        sentence_vectors.append(v)
    return sentence_vectors

In [ ]:
# calculate similarity of sentences
def calculate_similarities(clean_sentences,sentence_vectors):
    sim_mat = np.zeros([len(clean_sentences), len(clean_sentences)])           # create similarity matrix
    for i in range(len(clean_sentences)):
        for j in range(len(clean_sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]
    return sim_mat

In [ ]:
# use textrank algorithm to calculate similarity
def TextRank(sim_mat):
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    return scores

In [ ]:
data = pd.read_csv("../input/bbc-extractive-csv/bbc.csv")
print (data.shape)
data.head()

In [ ]:
articles = list(data['articles'])
summaries = list(data['summaries'])

In [ ]:
def generate_summary(text):
    sentences = clean_tokenize(text)                                                              # tokenize text
    sentences = list(filter(lambda a: a != "", sentences))                                        # remove empty string if any (Encode methods gives error if empty string is encountered)
    clean_sentences = pd.Series(sentences)                                                        # make a copy in pandas for processing
    clean_sentences = clean_sentences.apply(lambda s: s.translate(str.maketrans('', '', string.punctuation)))
    clean_sentences = [s.lower() for s in clean_sentences]                                        # make alphabets lowercase
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]                      # remove stopwords from the sentences
    sentence_vectors = convert_to_vectors(clean_sentences)                                        # create vectors for sentences
    sim_mat = calculate_similarities(clean_sentences,sentence_vectors)                            # calculate similarities between sentences
    scores = TextRank(sim_mat)                                                                    # applying textrank algorithm
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)      # sort sentences on the basis of similarity
    
    #-----Generating Summary-----#
    sn = 5                                                                                      # Specify number of sentences to form the summary
    summary = {}
    for i in range(sn):
        summary[sentences.index(ranked_sentences[i][1])] = ranked_sentences[i][1]
    
    final = ""
    for i in sorted (summary) : 
        final+= str(summary[i])+" "
    return final

In [ ]:
avg_rouge = [0]*9

In [ ]:
for i in range(500):  #range = len(articles)
    actual_summary = summaries[i]
    hypothesis = generate_summary(articles[i])
    
    print ("Actual Summary : ", end = "")
    print (actual_summary)
    print ("\n")
    print ("Predicted Summary : ", end = "")
    print (hypothesis)
    print ("\n")
    print ("Rouge Score : ", end = "")
    score = rouge.get_scores(hypothesis, actual_summary)
    print (score)
    
    dic = score[0]
    j = 0
    for k1,v1 in dic.items():
        for k2,v2 in v1.items():
            avg_rouge[j%9]+= v2
            j+=1
    
    print ("\n")
    print ("\n")

In [ ]:
n = len(avg_rouge)
for i in range(n):    
    avg_rouge[i]/=500    # divide by len(articles)

In [ ]:
print ("-------Rouge Scores-------")
print ("-----------with-----------")
print ("-----GLOVE Embeddings-----")
'''
print ("Rouge 1")
print ("f = ",avg_rouge[0])
print ("p = ",avg_rouge[1])
print ("r = ",avg_rouge[2])
print ("\n")
print ("Rouge 2")
print ("f = ",avg_rouge[3])
print ("p = ",avg_rouge[4])
print ("r = ",avg_rouge[5])
print ("\n")
'''
print ("\n")
print ("Rouge L")
print ("f = ",avg_rouge[6])
print ("p = ",avg_rouge[7])
print ("r = ",avg_rouge[8])